In [1]:
from llm.factory import LLMInterface
from setting.db import SessionLocal
from llm.embedding import get_text_embedding

# llm_client = LLMInterface("bedrock", "us.deepseek.r1-v1:0")
llm_client = LLMInterface("bedrock", "arn:aws:bedrock:us-east-1:841162690310:inference-profile/us.anthropic.claude-3-7-sonnet-20250219-v1:0")

In [2]:
from knowledgebase.doc_builder import DocBuilder

doc_builder = DocBuilder(llm_client, get_text_embedding)

In [ ]:
sections = doc_builder.split_markdown_by_heading("../graph/docs/programming-guides/proto3.md", {
    "doc_version": "1.0",
    "category": "Programming Guides",
})

In [ ]:
from knowledgebase.doc_model import KnowledgeBlock

from sqlalchemy.orm import joinedload

extend_blocks = []
with SessionLocal() as db:
    # Query for complete entities
    blocks = db.query(KnowledgeBlock).options(
        joinedload(KnowledgeBlock.source)
    ).all()
    

    # Now you can access all attributes
    for block in blocks:
        extend_blocks.append({
            'id': block.id,
            'name': block.name,
            'content': block.content,
            'source_content': block.source.content
        })

contexts = doc_builder.agument_block_context(extend_blocks)
contexts


In [5]:
from knowledgebase.doc_search import search_knowledge_blocks

search_knowledge_blocks("How to update protobuf file?", context_weight=0.6)

[('Generating Your Classes {#generating}', "## Generating Your Classes {#generating}\n\nTo generate the Java, Kotlin, Python, C++, Go, Ruby, Objective-C, or C# code\nthat you need to work with  ... (3091 characters truncated) ... ative to the\n    current directory, each file must reside in one of the `IMPORT_PATH`s so\n    that the compiler can determine its canonical name.\n", 0.49616014682773024, 0.4912048719407047, 0.49417803687292),
 ('File location {#location}', "## File location {#location}\n\nPrefer not to put `.proto` files in the same\ndirectory as other language sources. Consider\ncreating a subpackage `p ... (438 characters truncated) ... \n`//myteam/mypackage`.\n\nThe exception to this rule is when it's clear that the protos will be used only\nin a Java context, such as for testing.\n", 0.5525500028862514, 0.4686088058170843, 0.5189735240585845),
 ('default', '## default\n+++\ntitle = "Language Guide (proto 3)"\nweight = 40\ndescription = "Covers how to use the proto3 revi